<a href="https://colab.research.google.com/github/lkarjun/malayalam-language-model/blob/language-model/Malayalam-Language-Model/malayalam-language-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset Downloading

In [1]:
!pip install -qq dvc[gdrive]

!dvc get https://github.com/lkarjun/malayalam-language-model \
Datasets/

!unzip -q 'Datasets/*.zip' -d Datasets/

     |████████████████████████████████| 401 kB 5.4 MB/s 
     |████████████████████████████████| 41 kB 32 kB/s 
     |████████████████████████████████| 280 kB 34.5 MB/s 
     |████████████████████████████████| 217 kB 20.5 MB/s 
     |████████████████████████████████| 109 kB 54.3 MB/s 
     |████████████████████████████████| 44 kB 2.8 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 48 kB 4.9 MB/s 
     |████████████████████████████████| 133 kB 50.6 MB/s 
     |████████████████████████████████| 180 kB 55.9 MB/s 
     |████████████████████████████████| 287 kB 43.8 MB/s 
     |████████████████████████████████| 4.5 MB 46.7 MB/s 
     |████████████████████████████████| 548 kB 47.6 MB/s 
     |████████████████████████████████| 1.1 MB 42.3 MB/s 
     |████████████████████████████████| 3.6 MB 36.7 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |██████████████████████

## Malayalam Language Model

### Imports

In [11]:
# !pip install -Uqq fastai
from fastai.text.all import *

import pandas as pd

DATASET = Path("Datasets/")

### Dataset Loading

In [17]:
articles = pd.read_csv(DATASET/'article_files.csv')
magazines = pd.read_csv(DATASET/'magazine_files.csv')
wikitexts = pd.read_csv(DATASET/'wikitext_files.csv')

df = pd.concat([articles, magazines, wikitexts])

In [18]:
del articles, magazines, wikitexts